In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from utils.data_preprocess import *
from models import *

In [2]:
data = pd.read_csv('./data/ETHBTC-5m-data.csv')[:10000]
# data = data_split(data, 3)[0]

In [3]:
WINDOW_SIZE = 24
raw_data = DataLabeling(data, WINDOW_SIZE)
# train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data, train_size=0.6, val_size=0.2, test_size=0.2)
# Data = DataPreprocess(train_df, val_df, test_df, window_size=WINDOW_SIZE, label_size=1, label_columns=['Label'], shift=0, batch_size=8)

In [5]:
train_data = Data.train.prefetch(tf.data.AUTOTUNE)
val_data = Data.val.prefetch(tf.data.AUTOTUNE)

In [6]:
strong_model = CDT_1D_model(WINDOW_SIZE, 5).model

In [7]:
strong_model.fit(train_data,
                 epochs=1000,
                 validation_data=val_data,
                 steps_per_epoch=len(train_data),
                 validation_steps=int(0.15 * len(val_data)),
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2)])

Epoch 1/1000
11337/11337 [==============================] - 106s 9ms/step - loss: 0.4398 - accuracy: 0.8863 - val_loss: 0.3998 - val_accuracy: 0.8982 - lr: 0.0010
Epoch 2/1000
11337/11337 [==============================] - 98s 9ms/step - loss: 0.4336 - accuracy: 0.8863 - val_loss: 0.3997 - val_accuracy: 0.8982 - lr: 0.0010
Epoch 3/1000
 2913/11337 [======>.......................] - ETA: 1:14 - loss: 0.5114 - accuracy: 0.8551

In [16]:
strong_model.evaluate(Data.test)

473/473 [==============================] - 3s 6ms/step - loss: 0.4326 - accuracy: 0.8865


[0.43261733651161194, 0.8864508867263794]

In [18]:
weak_model = MLP_model(WINDOW_SIZE, 5).model
weak_model.fit(train_data,
                 epochs=1000,
                 validation_data=val_data,
                 steps_per_epoch=len(train_data),
                 validation_steps=int(0.15 * len(val_data)),
                 callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2),
                            create_model_checkpoint(model_name=weak_model.name, save_path='data_without_TIs')])

Epoch 1/1000
1418/1418 [==============================] - 16s 10ms/step - loss: 0.5291 - accuracy: 0.8858 - val_loss: 0.4028 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 2/1000
1418/1418 [==============================] - 16s 11ms/step - loss: 0.4456 - accuracy: 0.8863 - val_loss: 0.4019 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 3/1000
1418/1418 [==============================] - 15s 10ms/step - loss: 0.4359 - accuracy: 0.8863 - val_loss: 0.4014 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 4/1000
1418/1418 [==============================] - 15s 11ms/step - loss: 0.4331 - accuracy: 0.8863 - val_loss: 0.4009 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 5/1000
1418/1418 [==============================] - 14s 10ms/step - loss: 0.4321 - accuracy: 0.8863 - val_loss: 0.4006 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 6/1000
1418/1418 [==============================] - 16s 11ms/step - loss: 0.4319 - accuracy: 0.8863 - val_loss: 0.4004 - val_accuracy: 0.8980 - lr: 0.0010
Epoch 7/1000
1418/1418 [==========

In [19]:
weak_model.evaluate(Data.test)

473/473 [==============================] - 3s 7ms/step - loss: 0.4326 - accuracy: 0.8865


[0.432639479637146, 0.8864508867263794]

In [1]:
from tensorflow.keras import Sequential, layers, Model, Input
def model():
    input = Input(shape=[24, 5])
    x = layers.LSTM(300, return_sequences=True)(input)
    x = layers.LSTM(200, return_sequences=True)(x)
    x = layers.LSTM(100, return_sequences=True)(x)
    x = layers.LSTM(50, return_sequences=True)(x)
    x = layers.LSTM(20)(x)
    x = layers.Dense(500, activation='relu')(x)
    x = layers.Dropout(0.7)(x)
    output = layers.Dense(3, activation='softmax')(x)
    model = Model(input, output)
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
test = model()
test.fit(train_data, epochs=5)

Epoch 1/5
1418/1418 [==============================] - 48s 21ms/step - loss: 0.4419 - accuracy: 0.8861
Epoch 2/5
1418/1418 [==============================] - 32s 23ms/step - loss: 0.4377 - accuracy: 0.8863
Epoch 3/5
1418/1418 [==============================] - 30s 21ms/step - loss: 0.4360 - accuracy: 0.8863
Epoch 4/5
1418/1418 [==============================] - 31s 22ms/step - loss: 0.4353 - accuracy: 0.8863
Epoch 5/5
1418/1418 [==============================] - 34s 24ms/step - loss: 0.4347 - accuracy: 0.8863


# Backtest the strategy

In [4]:
from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy

In [5]:
data = raw_data.labelled_data

In [10]:
class Threshold(SignalStrategy):
    
    def init(self):
        super().init()
        self.label = self.data.Label
    
    def next(self):
        
        if self.label == True:
            self.buy()
        elif self.label == False:
            self.sell()
        else:
            pass

In [11]:
test = Backtest(data, Threshold, cash=10000, commission=0.0005)

In [12]:
test.run()

Start                     2017-07-14 04:00:00
End                       2017-08-17 21:15:00
Duration                     34 days 17:15:00
Exposure Time [%]                       99.98
Equity Final [$]                  3387.457659
Equity Peak [$]                  29459.104279
Return [%]                         -66.125423
Buy & Hold Return [%]              -65.977368
Return (Ann.) [%]                  -99.999996
Volatility (Ann.) [%]     1614524983611127.25
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.999992
Avg. Drawdown [%]                  -11.985652
Max. Drawdown Duration       29 days 23:20:00
Avg. Drawdown Duration        1 days 19:31:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                   22280.476428
Worst Trade [%]                    -64.011294
Avg. Trade [%]                    

In [13]:
test.plot()

Row(id='1854', ...)

In [14]:
data

,Open,High,Low,Close,Volume,Label
2017-07-14 04:00:00,0.339259,0.336937,0.351856,0.338078,0.000824,True
2017-07-14 04:05:00,0.339259,0.336960,0.351856,0.338100,0.003800,1
2017-07-14 04:10:00,0.339282,0.482926,0.351879,0.483820,0.005313,1
2017-07-14 04:15:00,0.459491,0.465134,0.467038,0.454854,0.060665,1
2017-07-14 04:20:00,0.459150,0.473802,0.470317,0.474711,0.006331,1
...,...,...,...,...,...,...
2017-08-17 20:55:00,0.117754,0.119916,0.129287,0.119213,0.118388,1
2017-08-17 21:00:00,0.120777,0.120692,0.130702,0.119691,0.072164,1
2017-08-17 21:05:00,0.123233,0.120167,0.129984,0.117163,0.070841,1
2017-08-17 21:10:00,0.121073,0.123406,0.132814,0.117551,0.110817,1
